<a href="https://colab.research.google.com/github/azhgh22/Walmart-Recruiting-Store-Sales-Forecasting/blob/main/notebooks/xgboost_base_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from google.colab import userdata
token = userdata.get('GITHUB_TOKEN')
user_name = userdata.get('GITHUB_USERNAME')
mail = userdata.get('GITHUB_MAIL')

!git config --global user.name "{user_name}"
!git config --global user.email "{mail}"
!git clone https://{token}@github.com/azhgh22/Walmart-Recruiting-Store-Sales-Forecasting.git

%cd Walmart-Recruiting-Store-Sales-Forecasting

Cloning into 'Walmart-Recruiting-Store-Sales-Forecasting'...
remote: Enumerating objects: 231, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 231 (delta 9), reused 2 (delta 2), pack-reused 209 (from 1)
Receiving objects: 100% (231/231), 6.68 MiB | 9.89 MiB/s, done.
Resolving deltas: 100% (106/106), done.
/content/Walmart-Recruiting-Store-Sales-Forecasting


In [3]:
from google.colab import userdata
!pip install -r requirements.txt
kaggle_json_path = userdata.get('KAGGLE_JSON_PATH')
! ./src/data_loader.sh -f {kaggle_json_path}

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 84.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 73.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 733.8/733.8 kB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
from src.config import *

stores = pd.read_csv(STORES_PATH)
features = pd.read_csv(FEATURES_PATH)
train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)

## **Merge tables and split data**

In [7]:
merged_train = pd.merge(train,stores,on='Store',how='left').merge(features,how='left',on=['Store','Date','IsHoliday'])

In [8]:
from src.time_series_split import TimeSeriesSplit
from src.config import SPLIT_DATE
merged_train.Date = pd.to_datetime(merged_train.Date)
x_train, x_val = TimeSeriesSplit(SPLIT_DATE).split(merged_train)
# for i in range(1,3):
#   x_train[f'shift{i}'] = x_train.groupby(['Store', 'Dept'])['Weekly_Sales'].shift(i)
#   x_val[f'shift{i}'] = x_val.groupby(['Store', 'Dept'])['Weekly_Sales'].shift(i)

y_train = x_train.pop('Weekly_Sales')
y_val = x_val.pop('Weekly_Sales')

In [9]:
merged_train.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday,Type,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment
0,1,1,2010-02-05,24924.50,False,A,151315,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106
1,1,1,2010-02-12,46039.49,True,A,151315,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106
2,1,1,2010-02-19,41595.55,False,A,151315,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106
3,1,1,2010-02-26,19403.54,False,A,151315,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106
4,1,1,2010-03-05,21827.90,False,A,151315,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106


# **Time series features**

In [18]:
import feature_engineering.time_features
import importlib
importlib.reload(feature_engineering.time_features)
from feature_engineering.time_features import FeatureAdder

# deal with **NaN** values

In [11]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer

class NaImputer(BaseEstimator, TransformerMixin):
  def __init__(self, strategy='mean'):
    self.strategy = strategy
    self.imputer = SimpleImputer(strategy=strategy)
    self.na_cols = []

  def fit(self, X, y=None):
    self.na_cols = [col for col in X.columns if X[col].isna().sum() > 0]
    self.imputer.fit(X[self.na_cols])
    return self

  def transform(self, X, y=None):
    x_copy = X.copy()
    x_copy[self.na_cols] = self.imputer.transform(x_copy[self.na_cols])
    return x_copy

# **Cat2Num**

In [12]:
from sklearn.base import BaseEstimator, TransformerMixin

class Cat2Num(BaseEstimator, TransformerMixin):
  def fit(self, X, y=None):
    return self

  def transform(self, X, y=None):
    x_copy = X.copy()
    x_copy.pop('Date')
    x_copy.IsHoliday = x_copy.IsHoliday.astype(int)
    x_copy.Type = x_copy.Type.astype('category').cat.codes
    return x_copy

# **Add lags to data**

# **XGboost**

In [13]:
from xgboost import DMatrix
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import mean_absolute_error
from src.utils import wmae

# **Grid search**

In [ ]:
from src.cross_validation import manual_model_search
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('adder', FeatureAdder()),
    ('imputer', NaImputer()),
    ('cat2num', Cat2Num()),
    # ('model', XGBRegressor(
    #     n_estimators=1000,
    #     learning_rate=0.1,
    #     max_depth=7,
    #     reg_lambda=3,
    #     min_split_loss=100,
    #     objective='reg:squarederror',
    #     random_state=42,
    # ))
  ])


mod = Pipeline([
    ('model', XGBRegressor(
        n_estimators=1000,
        learning_rate=0.1,
        max_depth=7,
        reg_lambda=3,
        min_split_loss=100,
        objective='reg:squarederror',
        random_state=42,
    ))
  ])


transformed_train = pipeline.fit_transform(x_train,y_train)
transformed_val = pipeline.transform(x_val)



param_grid = {
    'model__n_estimators': [200,500,800,1000],
    'model__learning_rate': [0.1],
    'model__max_depth': [7],
    'model__reg_lambda': [3],
    'model__min_split_loss': [100],
}


metric_kwargs = {
    'is_holiday': transformed_val['IsHoliday']
}

best_model, best_params, best_score = manual_model_search(
    model=mod,
    param_grid=param_grid,
    X_train=transformed_train,
    y_train=y_train,
    X_valid=transformed_val,
    y_valid=y_val,
    metric_func=wmae,
    metric_kwargs=metric_kwargs
)

print("\nBest Params:", best_params)
print("Best Validation Score:", best_score)

# **Model**

In [67]:
from sklearn.pipeline import Pipeline

pip = Pipeline([
    ('adder', FeatureAdder(
        add_week_num=True,
        add_holiday_flags=False,
        add_holiday_proximity=False,
        add_holiday_windows=False,
        add_fourier_features=True,
        add_month_and_year=False,
        list_of_holiday_proximity=None,
        # holiday_dates=HOLIDAY_DATES,
        replace_time_index = False,

        add_dummy_date=True,start_date=pd.Timestamp('2010-02-05'))),
    ('imputer', NaImputer()),
    ('cat2num', Cat2Num()),
    ('model', LagAdder(y_val,xgb,3))
  ])

In [68]:
transformed_train = pip.fit_transform(x_train)
transformed_val = pip.transform(x_val)

In [69]:
transformed_train.columns

Index(['Store', 'Dept', 'IsHoliday', 'Type', 'Size', 'Temperature',
       'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4',
       'MarkDown5', 'CPI', 'Unemployment', 'DateDummy', 'Month', 'Year',
       'WeekOfYear', 'week_sin', 'week_cos', 'month_sin', 'month_cos'],
      dtype='object')

In [125]:
# # from feature_engineering.lag_adder import LagAdder
# import feature_engineering.lag_adder
# import importlib
# importlib.reload(feature_engineering.time_features)
# from feature_engineering.lag_adder import LagAdder

In [27]:
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd
import numpy as np

class GroupMeanImputer(BaseEstimator, TransformerMixin):
    def __init__(self, group_cols=['Store', 'Dept'], target_cols=None, fallback=0):
        self.group_cols = group_cols
        self.target_cols = target_cols
        self.fallback = fallback
        self.group_means_ = None

    def fit(self, X, y=None):
        X = X.copy()
        if self.target_cols is None:
            self.target_cols = X.select_dtypes(include='number').columns[
                X.isna().any()
            ].tolist()

        self.group_means_ = (
            X.groupby(self.group_cols)[self.target_cols]
            .mean()
            .reset_index()
        )
        return self

    def transform(self, X):
        X = X.copy()
        for col in self.target_cols:
            # Merge group means
            means = self.group_means_[[*self.group_cols, col]]
            X = X.merge(means, on=self.group_cols, how='left', suffixes=('', '_group_mean'),sort=False)

            # Fill NaN with group mean, then fallback
            X[col] = X[col].fillna(X[f'{col}_group_mean'])
            X[col] = X[col].fillna(self.fallback)

            # Drop helper column
            X.drop(columns=[f'{col}_group_mean'], inplace=True)
        return X

from sklearn.impute import SimpleImputer

# class NaImputer(BaseEstimator, TransformerMixin):
#   def __init__(self, strategy='mean'):
#     self.strategy = strategy
#     self.imputer = SimpleImputer(strategy=strategy)
#     self.na_cols = []

#   def fit(self, X, y=None):
#     self.na_cols = [col for col in X.columns if X[col].isna().sum() > 0]
#     self.imputer.fit(X[self.na_cols])
#     return self

#   def transform(self, X, y=None):
#     x_copy = X.copy()
#     x_copy[self.na_cols] = self.imputer.transform(x_copy[self.na_cols])
#     return x_copy

class LagAdder(BaseEstimator, TransformerMixin):
  def __init__(self, val ,model ,lag_num:int = 2,date_col = 'DateDummy') -> None:
    super().__init__()
    self.lag_num = lag_num
    self.model = model
    self.y_val = val
    self.na_imputer = GroupMeanImputer()
    self.time = {}
    self.date_col = date_col

  def fit(self, x:pd.DataFrame, y:pd.DataFrame):
    x_ = x.copy()
    y_ = pd.DataFrame(y.copy())
    y_['Store'] = x_['Store']
    y_[self.date_col] = x_[self.date_col]
    y_['Dept'] = x_['Dept']
    for i in range(1,self.lag_num+1):
      x_[f'shift{i}'] = y_.groupby(['Store', 'Dept'])['Weekly_Sales'].shift(i)

    self.na_imputer.fit(x_)
    x_ = self.na_imputer.transform(x_)
    self.model.fit(x_,y)

    self.time = {}
    dates = sorted(x[self.date_col].unique())
    for i in dates:
      cur_data = y_[y_[self.date_col] == i][['Store', 'Dept','Weekly_Sales']].copy()
      self.time[i] = cur_data
    return self

  def transform(self, x:pd.DataFrame):
    answer = x.copy()
    answer['WeeklySales'] = 0.0
    # print(x)

    x_ = x.copy()

    dates = sorted(x[self.date_col].unique())

    time = self.time.copy()

    for i in dates:
      # print(i)
      cur_data = x_[x_[self.date_col] == i].copy()
      for j in range(1,self.lag_num+1):
        if time.get(i-j) is not None:
          cur_data[f'shift{j}'] = pd.merge(cur_data,time[i-j],how='left',on=['Store','Dept'],sort=False)['Weekly_Sales']
        else:
          cur_data[f'shift{j}'] = np.nan
          print(i)

      cur_data = self.na_imputer.transform(cur_data)
      # cur_data.fillna(500,inplace=True)
      # assert cur_data.isna().sum().sum() == 0
      pred = self.model.predict(cur_data)
      assert len(answer.loc[answer[self.date_col] == i,'WeeklySales'])==len(pred)
      answer.loc[answer[self.date_col] == i,'WeeklySales'] = pred.copy()
      t_i = x_.loc[x_[self.date_col] == i][['Store','Dept',self.date_col]]
      t_i['Weekly_Sales'] = pred.copy()
      time[i] = t_i

    return answer['WeeklySales']
    # return pd.DataFrame()

  def predict(self, x:pd.DataFrame):
    return self.transform(x)

In [28]:
from sklearn.pipeline import Pipeline

xgb = XGBRegressor(
        n_estimators=1000,
        learning_rate=0.01,
        max_depth=7,
        reg_lambda=1000,
        # min_split_loss=100,
        objective='reg:squarederror',
        random_state=42,
    )

pipeline = Pipeline([
    ('adder', FeatureAdder(
        add_week_num=True,
        add_holiday_flags=False,
        add_holiday_proximity=False,
        add_holiday_windows=False,
        add_fourier_features=True,
        add_month_and_year=False,
        list_of_holiday_proximity=None,
        # holiday_dates=HOLIDAY_DATES,
        replace_time_index = False,

        add_dummy_date=True,start_date=pd.Timestamp('2010-02-05'))),
    ('imputer', NaImputer()),
    ('cat2num', Cat2Num()),
    ('model', LagAdder(y_val,xgb,2))
  ])

model = pipeline.fit(x_train, y_train)


y_train_predict = model.predict(x_train)
y_val_predict = model.predict(x_val)

train_score = wmae(y_train, y_train_predict,x_train['IsHoliday'].to_list())
val_score = wmae(y_val, y_val_predict,x_val['IsHoliday'].to_list())
print(f"Train wmae: {train_score}, Val wmae: {val_score}")

0
0
1
Train wmae: 3001.246717622876, Val wmae: 3232.624473235746


In [40]:
model[-1].time[94]

,Store,Dept,DateDummy,Weekly_Sales
189431,20,16,94,40144.246094
363030,39,14,94,43158.023438
371621,40,5,94,47517.601562
236152,24,92,94,134678.843750
209720,22,20,94,9226.009766
...,...,...,...,...
244158,25,60,94,594.231384
180693,19,27,94,2766.245361
40027,5,2,94,12749.230469
3496,1,26,94,9533.174805


# **upload to WanDB**

In [29]:
! wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: azhgh22 (MLBeasts) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [30]:
import wandb
import joblib

y = merged_train['Weekly_Sales'].copy()
x = merged_train.drop(columns=['Weekly_Sales'])

fin_model = pipeline.fit(x,y)

joblib.dump(model, "xgb_pipeline_lags.pkl")
wandb.init(project="Walmart Recruiting - Store Sales Forecasting", name="xgboost:run2:lag_features")

wandb.config.update({
    'merge1' : 'train, store, how=left, on=Store',
    'merge2' : 'train, features, how=left, on=Store, Date, IsHoliday',
    'merged_tables' : ['train','stores','features'],
    'time_features' : [
        'DateDummy', 'Month', 'Year',
       'WeekOfYear', 'week_sin', 'week_cos', 'month_sin', 'month_cos'
    ],
    'lags' : 2,
    'add_dummy_date' : True,
    'start_date' : '2010-02-05',
    'score_metric' : 'WMAE',
    'score_policy' : {
        'weight on holidays' : 5,
        'weight on non_holidays' : 1
    },
    'model' : 'Xgboost',
    'n_estimators' : 1000,
    'learning_rate' : 0.1,
    'max_depth' : 7,
    'reg_lambda' : 1000,
    # 'min_split_loss' : 100,
    'objective' : 'reg:squarederror',
})

wandb.log({
    'train_wmae': train_score,
    'val_wmae': val_score
})


artifact = wandb.Artifact(
    name="xgb_pipeline",
    type="model",
    description="XGBoost pipeline with Date engineering and imputing"
)

artifact.add_file("xgb_pipeline_lags.pkl")
wandb.log_artifact(artifact)

wandb.finish()

wandb: Currently logged in as: azhgh22 (MLBeasts) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


train_wmae,▁
val_wmae,▁
train_wmae,3001.24672
val_wmae,3232.62447


# **Load model from wandb**

In [36]:
import wandb
import joblib

# Resume or init run
run = wandb.init(project="Walmart Recruiting - Store Sales Forecasting", name="xgboost:run2:lag_features")

# Download the artifact
artifact = run.use_artifact('MLBeasts/Walmart Recruiting - Store Sales Forecasting/xgb_pipeline:latest', type='model')
artifact_dir = artifact.download()

# Load the model
model = joblib.load(f"{artifact_dir}/xgb_pipeline_lags.pkl")
wandb.finish()

wandb:   1 of 1 files downloaded.  


In [37]:
test = pd.read_csv(TEST_PATH)
merged_test = test.merge(stores,on='Store',how='left').merge(features,how='left',on=['Store','Date','IsHoliday'])
# merged_test.Date = pd.to_datetime(merged_test.Date)
model.predict(merged_test)

,WeeklySales
0,27499.441406
1,9485.120117
2,28529.046875
3,23762.226562
4,12615.354492
...,...
115059,584.115540
115060,657.016357
115061,548.248718
115062,552.254272


In [110]:
merged_test

,Store,Dept,Date,IsHoliday,Type,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment
0,1,1,2012-11-02,False,A,151315,55.32,3.386,6766.44,5147.70,50.82,3639.90,2737.42,223.462779,6.573
1,1,1,2012-11-09,False,A,151315,61.24,3.314,11421.32,3370.89,40.28,4646.79,6154.16,223.481307,6.573
2,1,1,2012-11-16,False,A,151315,52.92,3.252,9696.28,292.10,103.78,1133.15,6612.69,223.512911,6.573
3,1,1,2012-11-23,True,A,151315,56.23,3.211,883.59,4.17,74910.32,209.91,303.32,223.561947,6.573
4,1,1,2012-11-30,False,A,151315,52.34,3.207,2460.03,NaN,3838.35,150.57,6966.34,223.610984,6.573
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115059,45,98,2013-06-28,False,B,118221,76.05,3.639,4842.29,975.03,3.00,2449.97,3169.69,NaN,NaN
115060,45,98,2013-07-05,False,B,118221,77.50,3.614,9090.48,2268.58,582.74,5797.47,1514.93,NaN,NaN
115061,45,98,2013-07-12,False,B,118221,79.37,3.614,3789.94,1827.31,85.72,744.84,2150.36,NaN,NaN
115062,45,98,2013-07-19,False,B,118221,82.84,3.737,2961.49,1047.07,204.19,363.00,1059.46,NaN,NaN
